# 환경설정

In [6]:
# TextMining_수만휘_수시게시판_통합.csv에서 양식별로 column 찢기
import pandas as pd
import numpy as np
import re

# 데이터 로드
df = pd.read_csv('TextMining_수만휘_수시게시판_통합.csv',encoding='utf-8-sig')
df['content'] = df['content'].astype(str)
df['content'] = df['content'].str.strip()

# 데이터프레임 생성
new_df = pd.DataFrame(df, columns=df.columns)

# 새로운 열을 추가하여 데이터를 분류
new_columns = ['1. 입시결과 요약', '2. 출신고교 종류', '3. 내신/수능점수', '4. 스펙', '5. 후배들에게 전하고 싶은 말', '6. 수험 수기']
new_df = pd.DataFrame(columns=new_columns)

current_col_index = 0  # 현재 열 인덱스
row_data = [''] * 6  # 초기화된 분류 데이터 리스트

for index, row in df.iterrows():
    content = str(row['content']).replace('\n', ' ')  # 개행 문자 제거
    
    # 개행 문자 제거
    content = re.sub(r'[^\w\s\.\-/.~]', '', content)
    
    sentences = re.split(r'(?<=\d)\.(?=\s)', content)
    
    row_data = [''] * 6  # 각 행마다 row_data를 새로 초기화
    
    for sentence in sentences:
        # 문장의 맨 끝에 있는 1개의 숫자 제거
        sentence = re.sub(r'\d$', '', sentence)
        
        if '입시결과' in sentence:
            current_col_index = 0  # '1'이 나왔을 때 현재 열 인덱스를 0으로 설정
            row_data[current_col_index] = sentence.replace('입시결과 요약합격 또는 불합격한 대학명/ 학과 / 전형', '')  # 1. 열에 해당하는 데이터에 문장 추가
        elif '출신고교' in sentence:
            current_col_index = 1  # '2'가 나왔을 때 현재 열 인덱스를 1로 설정
            row_data[current_col_index] = sentence.replace('출신고교의 종류 / 성별성별이 영향을 주는 전형이 아니면 적지 않으셔도 됩니다', '')  # 2. 열에 해당하는 데이터에 문장 추가
        elif '내신/수능' in sentence:
            current_col_index = 2  # '3'이 나왔을 때 현재 열 인덱스를 2로 설정
            row_data[current_col_index] = sentence.replace('내신/수능 점수', '')  # 3. 열에 해당하는 데이터에 문장 추가
        elif '스펙' in sentence:
            current_col_index = 3  # '4'가 나왔을 때 현재 열 인덱스를 3로 설정
            row_data[current_col_index] = sentence.replace('스펙', '')  # 4. 열에 해당하는 데이터에 문장 추가
        elif '후배들에게' in sentence:
            current_col_index = 4  # '5'가 나왔을 때 현재 열 인덱스를 4로 설정
            row_data[current_col_index] = sentence.replace('후배들에게 전하고 싶은 말', '')  # 5. 열에 해당하는 데이터에 문장 추가
        elif '수험 수기' in sentence:
            current_col_index = 5  # '6'이 나왔을 때 현재 열 인덱스를 5로 설정
            row_data[current_col_index] = sentence.replace('수험 수기작성하시고 픈 분만 작성하셔도 됩니다.', '')  # 6. 열에 해당하는 데이터에 문장 추가
        else:
            row_data[current_col_index] += sentence  # 현재 열에 해당하는 데이터에 문장 추가
    
    new_df.loc[index] = row_data

# 기존 DataFrame에 열 추가
df[new_columns] = new_df[new_columns]

# '1. 입시결과'나 '2. 스펙' 열 등 데이터가 없는 행을 삭제하는 코드
df = df.drop(df[(df['1. 입시결과 요약'] == '') | (df['1. 입시결과 요약'] == ' ') | (df['2. 출신고교 종류'] == '') | (df['2. 출신고교 종류'] == ' ') | (df['3. 내신/수능점수'] == '') | (df['3. 내신/수능점수'] == ' ') | (df['4. 스펙'] == '') | (df['4. 스펙'] == ' ') | (df['5. 후배들에게 전하고 싶은 말'] == '') | (df['5. 후배들에게 전하고 싶은 말'] == ' ')].index)
df.drop('6. 수험 수기', axis=1, inplace=True)
df.to_csv('수만휘_전처리.csv', index=False, encoding='utf-8-sig')

print('분류된 데이터가 성공적으로 저장되었습니다.')

분류된 데이터가 성공적으로 저장되었습니다.


In [7]:
import pandas as pd
import re

# 엑셀 파일 경로
file_path = '수만휘_전처리.csv'

# 엑셀 파일 읽기
df = pd.read_csv(file_path)

# 9열에만 전처리를 적용할 경우
target_column_school = 9
# 10열에만 전처리를 적용할 경우
target_column_score = 10

# 전처리를 위한 함수 정의 10열
def preprocess_score(value):
    if isinstance(value, str):
        # "평균", "총", "전과목", "최종" 처리
        words = value.split()
        result1 = []
        for i, word in enumerate(words):
            if word in ['평균', '총', '전과목', '최종', '전교과', '내신', '총내신', '총합']:
                if i+1 < len(words):
                    number = re.findall(r'\d+\.\d+', words[i+1])
                    result1.extend(number)

        # 숫자5개 연속 처리
        for word in words:
            if len(word) == 5 and word.isdigit():
                result1.append(word)

        return ', '.join(result1) if result1 else ''
    return ''

# 전처리를 위한 함수 정의 9열
def preprocess_school(value):
    if isinstance(value, str):
        words = value.split()  # 문장을 단어로 분리
        result2 = []
        for word in words:
            if any(keyword in word for keyword in ['일반', '과학', '특성화', '자사', '외','외국어']):
                if '일반' in word:
                    word = '일반고'
                elif '과학' in word:
                    word = '과학고'
                elif '특성화' in word:
                    word = '특성화고'
                elif '자사' in word:
                    word = '자사고'
                elif '외' in word or '외국어' in word:
                    word = '외고'
                result2.append(word)
        if result2:
            return ', '.join(result2)
    return ''


column_name1 = df.columns[target_column_score]
column_name2 = df.columns[target_column_school]

# 전처리된 값을 저장할 새로운 열 이름
new_column_name1 = "preprocessed_score"
new_column_name2 = "preprocessed_school"

# 새로운 열 생성 및 전처리된 값 저장
#df[new_column_name1] = df[column_name1].apply(preprocess_score)
#df[new_column_name2] = df[column_name2].apply(preprocess_school)
df['3. 내신/수능점수'] = df['3. 내신/수능점수'].apply(preprocess_score)
df['2. 출신고교 종류'] = df['2. 출신고교 종류'].apply(preprocess_school)

# 새로운 CSV 파일로 저장
df.to_csv('수만휘_전처리_내신&학교.csv', index=False, encoding='utf-8-sig')
print("추출이 완료되었습니다.")


추출이 완료되었습니다.


In [8]:
# 수능 점수 제거 = 내신 점수만 남김
# 이후 평균 내기 위해서. 그리고 애초에 여기는 "수시 합격 수기 게시판"이라 내신 점수만 있어도 된다고 판단.
import pandas as pd
import re
import numpy as np

# CSV 파일 불러오기
data = pd.read_csv('수만휘_전처리_내신&학교.csv')

# 정규식 패턴으로 3개 이상 연속된 숫자 형태의 값 찾기
pattern = r'\d{3,}'
data['3. 내신/수능점수'] = data['3. 내신/수능점수'].apply(lambda x: re.sub(pattern, '', str(x).replace(',', '')) if pd.notnull(x) else np.nan)

# 결과를 새로운 CSV 파일로 저장하기
data.to_csv('수만휘_전처리_내신&학교.csv', index=False, encoding='utf-8-sig')


In [9]:
import pandas as pd
import re

# CSV 파일 불러오기
csv_file = "수만휘_전처리_내신&학교.csv"
df = pd.read_csv(csv_file)

def extract_string_with_words(sentence, conditions):
    matches = []
    for condition in conditions:
        word1 = condition["word1"]
        word2 = condition["word2"]
        pattern = r"\b" + word1 + r"\s*(?:\d+\s*)?\w*\s*" + word2 + r"\b"  # 수정된 정규표현식
        match = re.search(pattern, sentence, flags=re.IGNORECASE)  # 대소문자 무시하도록 flags 추가
        if match:
            extracted_string = match.group().strip()  # 매치된 문자열에서 앞뒤 공백 제거
            matches.append(extracted_string)
    return matches

# 추출할 단어 설정
conditions = [
    {"word1": "생기부", "word2": "장"},
    {"word1": "독서", "word2": "권"},
    {"word1": "봉사활동", "word2": "시간"},
    {"word1": "봉사", "word2": "시간"},
    {"word1": "상장", "word2": "개"},
    {"word1": "수상", "word2": "개"}
]

# 추출 작업 수행 및 결과 저장
target_column = 11  # 추출할 열의 인덱스 (0부터 시작)
column_name = df.columns[target_column]  # 추출할 열의 이름
extracted_data = []
for condition in conditions:
    word1 = condition["word1"]
    word2 = condition["word2"]
    extracted_values = df[column_name].apply(lambda x: extract_string_with_words(str(x), conditions))
    extracted_data.append(extracted_values)

# 추출한 값들을 기존 데이터프레임에 새로운 열로 추가
# for i, condition in enumerate(conditions):
#     word1 = condition["word1"]
#     word2 = condition["word2"]
#     new_column_name = f"{word1}_{word2}"
#     df[new_column_name] = extracted_data[i]
new_column_name = "스펙_핵심_추출"
df[new_column_name] = extracted_data[0]

# 수정된 데이터프레임을 CSV 파일로 출력하여 저장
output_csv_file = "수만휘_전처리_내신&학교&스펙.csv"
df.to_csv(output_csv_file, index=False, encoding='utf-8-sig')
print("출력 파일이 생성되었습니다.")


출력 파일이 생성되었습니다.


In [10]:
import csv
import os

def count_words_in_csv(input_file, target_column_index, words):
    with open(input_file, 'r', encoding='utf-8') as file:
        reader = csv.reader(file)
        rows = list(reader)

    counts = []
    for row in rows:
        cell = row[target_column_index]
        count = 0
        for word in words:
            count += cell.count(word)
        counts.append(count)

    output_file = os.path.splitext(os.path.basename(input_file))[0] + '_output.csv'

    with open(output_file, 'w', encoding='utf-8', newline='') as file:
        writer = csv.writer(file)
        for row, count in zip(rows, counts):
            row.append(str(count))
            writer.writerow(row)

input_file = '수만휘_전처리_내신&학교&스펙.csv'
target_column_index = 11
words = ['동아리', '상장', '수상', '독서', '봉사', '반장', '회장', '자소서', '학생회', '생기부', '세특', '임원', '프로젝트', '활동', '원서']

count_words_in_csv(input_file, target_column_index, words)


In [11]:
!pip install summarize

ERROR: Could not find a version that satisfies the requirement summarize (from versions: none)
ERROR: No matching distribution found for summarize


In [12]:
!pip install summa

In [13]:
!pip install summa.summarizer

ERROR: Could not find a version that satisfies the requirement summa.summarizer (from versions: none)
ERROR: No matching distribution found for summa.summarizer


In [14]:
import pandas as pd
from summa.summarizer import summarize

df = pd.read_csv('수만휘_전처리_내신&학교&스펙_output.csv', encoding='utf-8-sig')

# 열 이름 변경
df = df.rename(columns={'0': '중요단어 카운팅 수'})

# 텍스트 요약
summary = []
for text in df['4. 스펙']:
    if isinstance(text, str):  # 문자열인지 확인
        words = text.split()  # 단어를 공백 기준으로 분리
        num_words = len(words)
        if num_words >= 20:  # 20단어 이상인 경우 요약 진행
            summarized_text = summarize(text, words=30)  # 30단어로 요약
            summarized_text = summarized_text.replace('\n', ' ').replace('/', ' ')
        else:
            summarized_text = text.replace('\n', ' ').replace('/', ' ')  # 20단어 이하인 경우 원문 그대로 출력
    else:
        summarized_text = ''  # 빈 데이터나 문자열이 아닌 경우 처리

    summary.append(summarized_text)

# 요약 결과를 새로운 열로 추가
df['4. 요약'] = summary

df.to_csv('수만휘_전처리_내신&학교&스펙&요약.csv', index=False, encoding='utf-8-sig')


In [15]:
import pandas as pd
from summa.summarizer import summarize

# 데이터 로드
df = pd.read_csv('수만휘_전처리_내신&학교&스펙&요약.csv', encoding='utf-8-sig')

# '5. 후배들에게 전하고 싶은 말' 열의 텍스트 요약
summary = []
for text in df['5. 후배들에게 전하고 싶은 말']:
    if isinstance(text, str):  # Check if the text is a string
        words = text.split()  # 단어를 공백 기준으로 분리
        num_words = len(words)
        if num_words >= 20:  # 20단어 이상인 경우 요약 진행
            summarized_text = summarize(text, words=20)  # 20단어로 요약
            summarized_text = summarized_text.replace('\n', ' ').replace('/', ' ')
        else:
            summarized_text = text.replace('\n', ' ').replace('/', ' ')  # 20단어 이하인 경우 원문 그대로 출력
    else:
        summarized_text = ''  # Handle empty or non-string data

    summary.append(summarized_text)

# 요약 결과를 새로운 열로 추가
df['5. 요약'] = summary

df.to_csv('수만휘_전처리_내신&학교&스펙&요약.csv', index=False, encoding='utf-8-sig')

여기까지는 맨 처음 각자 전 처리 한 코드 합친 거.

[NEW]대학 분류 코드. 열을 여러개 생성하는 게 아니라, row를 복사해서 나머진 똑같고 '입시결과' 열의 내용만 달라지게 수정

In [16]:
# 줄넘김으로 대학 분류하는 코드. 기존 열 추가 코드에서 행 복사->입시결과 열 값만 다르게 넣는 걸로 수정함.

import pandas as pd

# CSV 파일 불러오기
data = pd.read_csv('수만휘_전처리_내신&학교&스펙&요약.csv')

# "1. 입시결과 요약" 열에서 텍스트 끊어서 "new_col"에 저장
results = []
summary_column = data['1. 입시결과 요약']
for text in summary_column:
    if pd.notnull(text):  # NaN 값이 아닌 경우에만 처리
        # 대나 대학교가 포함된 단어 앞에서 줄넘김 처리
        lines = []
        current_line = ''
        for word in text.split():
            if '대' in word or '대학교' in word:
                if current_line:
                    lines.append(current_line)
                current_line = word
            else:
                current_line += ' ' + word
        if current_line:
            lines.append(current_line)
        formatted_text = '\n'.join(lines)
        # 맨 마지막 줄에 줄넘김 추가
        if formatted_text:
            formatted_text += '\n'
        results.append(formatted_text)
    else:
        results.append('')  # 결측치인 경우 빈 문자열로 처리

# "new_col" 열을 DataFrame에 추가
data['new_col'] = results

# 'new_col'열이 결측치인 경우, 해당 row 제거
data = data[data['new_col'] != '']

# 새로운 칼럼 생성
max_columns = 10  # 최대 칼럼 개수 제한
num_lines = data['new_col'].str.count('\n')  # 줄넘김 개수
num_lines[num_lines > max_columns] = max_columns  # 최대 칼럼 개수 제한
new_columns = [f"입시결과{i+1}" for i in range(max_columns)]  # 칼럼 이름 생성

# 값 저장
new_data = pd.DataFrame()
for i, row in data.iterrows():
    for j in range(max_columns):
        new_row = row.copy()
        if j < num_lines[i]:
            new_row['입시결과'] = row['new_col'].split('\n')[j]
        else:
            new_row['입시결과'] = ''
        new_data = pd.concat([new_data, new_row.to_frame().T], ignore_index=True)

# '입시결과'열이 결측치인 경우, 해당 row 제거
#new_data = new_data.dropna(subset=['입시결과'])
# 결측치인 row 제거
new_data = new_data[new_data['입시결과'] != '']

# 새로운 CSV 파일로 저장
new_data.to_csv('수만휘_전처리_내신&학교&스펙&요약&대학분류.csv', index=False, encoding='utf-8-sig')

[대학 나누기 + 단어 태깅까지 완료한 코드]

In [17]:
import pandas as pd
import re

# CSV 파일 불러오기
df = pd.read_csv('수만휘_전처리_내신&학교&스펙&요약&대학분류.csv')

# 태깅 결과를 저장할 리스트 초기화
universities = []
departments = []
results = []

# 각 텍스트에서 '대학교', '학과명', '합불여부' 태깅
for text in df['입시결과']:
    # 태깅 결과를 저장할 딕셔너리 초기화
    tags = {
        '대학명': None,
        '학과명': None,
        '합불여부': None
    }

    # '대학교' 태깅
    match = re.search(r'\b(\w*대학\w*|.*대)\b', text)
    if match:
        tags['대학명'] = match.group()

    # '학과명' 태깅
    match = re.search(r'\b(\w*과\w*|\w*학과\w*|\w*학부\w*)\b', text)
    if match:
        department = match.group()
        if '교과' not in department and '종합' not in department:
            tags['학과명'] = department

    # '합불여부' 태깅
    match = re.search(r'\b(\w*합(?!.*[대학교|학과명])\w*)|(\w*탈(?!.*[대학교|학과명])\w*)|(\w*불(?!.*[대학교|학과명])\w*)|(\w*예(?!.*[대학교|학과명])\w*)|(\w*최(?!.*[대학교|학과명])\w*)|(\w*미(?!.*[대학교|학과명])\w*)|(\w*떨(?!.*[대학교|학과명])\w*)\b', text)
    if match:
        department = match.group()
        if '종합' not in department:
            tags['합불여부'] = department

    # 태깅 결과를 리스트에 저장
    universities.append(tags['대학명'])
    departments.append(tags['학과명'])
    results.append(tags['합불여부'])

# 태깅 결과로 새로운 열을 추가한 데이터프레임 생성
df['대학명'] = universities
df['학과명'] = departments
df['합불여부'] = results

# 새로운 CSV 파일로 저장
df.to_csv('수만휘_전처리_내신&학교&스펙&요약&대학분류&태깅.csv', index=False, encoding='utf-8-sig')


최종 검색 모델 만들기
-현재 '대학명','학과명'치면 해당하는 row까지 검색 가능


'광운대학교' 라고 검색하는 것보다 '광운'이라고 검색해야 더 잘 나옵니다. 해당 단어 포함하고 있는 row을 검색하게 만들었기 때문데, 광운대학교라고 치면 "광운대학교"라는 단어가 그대로 박힌 row밖에 안 뽑아줘요. 
다른 학교로 생각하셔도 똑같아요. 고려대 검색하려면 "고려/경영" 또는 "고려/경영학과" 이런 식으로 검색하시는 게 좋아용

In [18]:
# 결측치 확인하는 코드

# import pandas as pd

# # CSV 파일 불러오기
# data = pd.read_csv('수만휘_전처리_내신&학교&스펙&대학분류&태깅.csv')
# 원하는 col 결측치 보기
missing_values = df['학과명'].isnull().sum()
print(f"결측치 개수: {missing_values}")
# 전체 col 갯수 보기
total_rows = len(df)
print(f"전체 row 개수: {total_rows}")

#칼럼 Name 출력
column_names = df.columns
print(column_names)

결측치 개수: 5890
전체 row 개수: 14919
Index(['no', 'title', 'link', 'name', 'date', 'view', 'content', 'idx_no',
       '1. 입시결과 요약', '2. 출신고교 종류', '3. 내신/수능점수', '4. 스펙', '5. 후배들에게 전하고 싶은 말',
       '스펙_핵심_추출', '중요단어 카운팅 수', '4. 요약', '5. 요약', 'new_col', '입시결과', '대학명',
       '학과명', '합불여부'],
      dtype='object')


In [19]:
print(filtered_df['3. 내신/수능점수'])

NameError: name 'filtered_df' is not defined

In [1]:
!pip install Selenium
!apt-get update 
!apt install chromium-chromedriver

'apt-get'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.
'apt'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [2]:
pip install konlpy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


★★★검색 모델

In [3]:
# 여기가 "학교명/학과명"으로 검색하는 기능 구현 코드

import pandas as pd
import re

# CSV 파일 불러오기
df = pd.read_csv('수만휘_전처리_내신&학교&스펙&요약&대학분류&태깅.csv')

# 사용자 입력 받기
user_input = input("원하시는 대학명과 학과를 입력해주세요 ex)광운/정보융합 : ")

# 대학명과 학과명 추출
match = re.search(r'(\w+)/(\w+)', user_input)
if match:
    university = match.group(1)
    department = match.group(2)
else:
    print("입력이 올바르지 않습니다.")
    exit()

# 대학명과 학과명이 포함된 행 찾기
filtered_df = df[df['대학명'].str.contains(university) & df['학과명'].str.contains(department)]

# 결과 출력
if len(filtered_df) > 0:
    print("태깅된 행을 찾았습니다. 해당하는 열의 개수:", len(filtered_df))
    print("")
else:
    print("일치하는 태깅된 행이 없습니다.")

##########################################################################################################
# 여기부터 희원이 키워드 추출 코드
import pandas as pd
from konlpy.tag import Okt
from collections import Counter
import re
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import normalize
import numpy as np

# CSV 파일 불러오기
df = pd.read_csv('수만휘_전처리_내신&학교&스펙&대학분류&태깅.csv', encoding='utf-8-sig')

filtered_df.loc[:, '4. 스펙'] = filtered_df['4. 스펙'].fillna('').apply(lambda x: re.sub('[^가-힣0-9\s]', '', x))

# '4. 스펙' 열의 내용을 하나의 문자열로 합치기
spec_string = ' '.join(filtered_df['4. 스펙'].astype(str).tolist())

# 형태소 분석기 초기화
okt = Okt()
# 형태소 분석 수행
morphs = okt.nouns(spec_string)
# 단어 카운트
word_counter = Counter(morphs)

# 가장 많이 나온 상위 10개의 단어 추출
top_10_words = word_counter.most_common(10)
top_10_words = [word[0] for word in top_10_words]
# 상위 10개 단어
sentence = ' '.join(top_10_words)

#입력한 단어 리스트(words)에 해당하는 문장들을 filtered_df의 '4. 스펙' 열에서 추출하는 작업을 수행
# 입력한 단어 리스트
words = ['동아리', '상장', '수상', '봉사', '반장', '회장', '자소서', '학생회', '세특', '임원', '프로젝트', '활동', '원서', '상', '대회', '부장', '확장', '활동', '발표', '주제', '분야', '수상경력', '보고서']

# 단어 리스트에 해당하는 문장 추출
selected_sentences = []
for sentence in filtered_df['4. 스펙']:
    if any(word in sentence for word in words):
        sentence = sentence.replace('\n', '.').replace('  ', ' ')
        selected_sentences.append(sentence)
        
sentences = selected_sentences
# 문장들을 담은 리스트로부터 데이터프레임 생성
spec_data = pd.DataFrame({'Sentences': sentences})

# 부산대 맞춤법 검사기 - 1차 교정
# 브라우저 주소를 통해 "부산대 맞춤법 교정기" 페이지로 이동해서 교정을 시행한 후, 교정된 텍스트을 가져오는 코드
from selenium import webdriver
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
def spell_checking(sentence, text_list):
    for i in (range(len(sentence))):
        time.sleep(0.5)
        wd.get('http://speller.cs.pusan.ac.kr/')
        try:
            wd.find_element_by_xpath('//*[@id="text1"]').send_keys(sentence[i])
            wd.find_element_by_xpath('//*[@id="btnCheck"]').click()
            time.sleep(1)
            entity_num = 0
            while True:
                try:
                    wd.find_element_by_xpath('//*[@id="tdReplaceWord_'+str(entity_num)+'"]/ul/li/a').click()
                    entity_num += 1
                except:
                    break
            texts = wd.find_element_by_xpath('//*[@id="tdCorrection1stBox"]').text
            text_list.append(texts)
        except:
            text_list.append(sentence[i])

test_spell_list = []

spell_checking(spec_data['Sentences'], test_spell_list)

# 문장을 문장단위로 나누는 함수
# test_spell_list에는 맞춤법 교정이 적용된 문장들이 저장되어 있음
def split_sentences(text):
    sentences = text.split('.')
    cleaned_sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return cleaned_sentences

# 문장별로 나누기
sentences = []
for text in test_spell_list:
    sentences.extend(split_sentences(text))
    
twitter = Twitter()

#불용어제거
stopwords = ['학년' , "때", "개", "권", "시간", "나", "우리", "저희", "따라", "의해", "을", "를", "에", "의", "가", "장"]

def get_nouns(sentences):
    nouns = []
    for sentence in sentences:
        if sentence is not '':
            nouns.append(' '.join([noun for noun in twitter.nouns(str(sentence))
                                  if noun not in stopwords and len(noun) > 1]))
    return nouns

nouns = get_nouns(sentences)

# TF-IDF 모델 생성 및 그래프 생성
tfidf = TfidfVectorizer()
cnt_vec = CountVectorizer()
graph_sentence = []

def build_sent_graph(sentence):
    tfidf_mat = tfidf.fit_transform(sentence).toarray()
    graph_sentence = np.dot(tfidf_mat, tfidf_mat.T)
    return graph_sentence

sent_graph = build_sent_graph(nouns)

def build_words_graph(sentence):
    cnt_vec_mat = normalize(cnt_vec.fit_transform(sentence).toarray().astype(float), axis=0)
    vocab = cnt_vec.vocabulary_
    return np.dot(cnt_vec_mat.T, cnt_vec_mat), {vocab[word] : word for word in vocab}

words_graph, idx2word = build_words_graph(nouns)

# TextRank 구현
# 각 단어의 중요도를 계산하여 반환
def get_ranks(graph, d=0.85): # d = damping factor
    A = graph
    matrix_size = A.shape[0]
    for id in range(matrix_size):
        A[id, id] = 0 # diagonal 부분을 0으로
        link_sum = np.sum(A[:,id]) # A[:, id] = A[:][id]
        if link_sum != 0:
            A[:, id] /= link_sum
        A[:, id] *= -d
        A[id, id] = 1

    B = (1-d) * np.ones((matrix_size, 1))
    ranks = np.linalg.solve(A, B) # 연립방정식 Ax = b
    return {idx: r[0] for idx, r in enumerate(ranks)}

sent_rank_idx = get_ranks(sent_graph)  #sent_graph : sentence 가중치 그래프

sorted_sent_rank_idx = sorted(sent_rank_idx, key=lambda k: sent_rank_idx[k], reverse=True)

#앞에 있는 인덱스의 문장일수록 점수가 가장 높게 매겨졌음을 나타냄.
word_rank_idx = get_ranks(words_graph)
sorted_word_rank_idx = sorted(word_rank_idx, key=lambda k: word_rank_idx[k], reverse=True)

def summarize(sent_num=5):
    summary = []
    index = []
    for idx in sorted_sent_rank_idx[:sent_num].copy():
        index.append(idx)

    index.sort()

    for idx in index:
        summary.append(sentences[idx])

    return summary

def find_no_values(summaries, df):
    no_values = []
    for summary in summaries:
        found = False  # 요약문이 발견되었는지 여부를 나타내는 변수
        for index, row in df.iterrows():
            if summary in row['4. 스펙']:
                no_values.append(row['no'])
                found = True
                break
        if not found:
            no_values.append(None)  # 요약문이 발견되지 않은 경우 None 값을 추가
    return no_values


# summarize() 함수를 사용하여 요약문 생성
# summaries = summarize()

# 요약문과 해당하는 "no" 값을 출력
# no_values = find_no_values(summaries, filtered_df)
# for i in range(len(summaries)):
#     print("요약문:", summaries[i])
#     print("해당하는 'no' 값:", no_values[i])
#     print()
# summarize() 함수를 사용하여 요약문 생성
summaries = summarize()

# 요약문과 해당하는 "no" 값을 출력하지 않음
no_values = find_no_values(summaries, filtered_df)
for i in range(len(summaries)):
    pass

def keywords(word_num=10):
    keywords = []
    index=[]
    for idx in sorted_word_rank_idx[:word_num]:
        index.append(idx)

    #index.sort()
    for idx in index:
        keywords.append(idx2word[idx])

    print(f"{user_input} 사람들의 스펙 핵심 키워드 : ")
    print(keywords)
# keywords()

##########################################################################################################
# 이건 지원이가 짠 3. 내신/수능 점수 열에서 평균 구하는 코드
import pandas as pd
import re
import numpy as np

# 숫자 값 추출 함수 정의
def extract_number(value):
    try:
        number = re.findall(r'[+-]?\d+(?:\.\d+)?', str(value))
        if len(number) > 0:
            return float(number[0])
        else:
            return np.nan
    except ValueError:
        return np.nan

# 각 셀에 있는 숫자 값 추출하여 리스트에 저장
numbers = []
for value in filtered_df['3. 내신/수능점수']:
    number = extract_number(value)
    if not np.isnan(number):
        numbers.append(number)

# 평균 계산
average = np.mean(numbers)

# 평균 값을 소수점 둘째 자리에서 반올림하여 출력
# rounded_average 변수에 검색한 학교/학과 사람들의 평균 내신 점수 넣음.
rounded_average = round(average, 2)
# print("평균:", rounded_average)

# 여기서부터는 filtered_df에 대한 고등학교 비율 계산 코드
import pandas as pd

# filtered_df 데이터프레임에서 학교 유형 분류 및 카운트
school_types = ['일반고', '자사고', '특성화고', '과학고', '외고']
total_count = len(filtered_df)
type_counts = filtered_df['2. 출신고교 종류'].value_counts()

# '외고'와 '외국어고'를 하나로 합치기
type_counts['외고'] = type_counts.get('외고', 0) + type_counts.get('외국어고', 0)
if '외국어고' in type_counts:
    del type_counts['외국어고']

##########################################################################################################
# 검색 결과 출력 창
# '중요단어 카운팅 수'기반 [결과 출력 창]
import pandas as pd
import numpy as np


# '중요단어 카운팅 수'에 대한 구간 설정
bins = np.arange(0, filtered_df['중요단어 카운팅 수'].max() + 6, 5)
# 구간별로 치환된 값을 새로운 열에 저장
filtered_df['가중치_중요단어카운팅수'] = pd.cut(filtered_df['중요단어 카운팅 수'], bins, labels=False) + 1

# 'view' 열의 값을 숫자형으로 변환
filtered_df['view'] = pd.to_numeric(filtered_df['view'], errors='coerce')

# '3. 내신/수능점수' 열의 값을 숫자형으로 변환
filtered_df['3. 내신/수능점수'] = pd.to_numeric(filtered_df['3. 내신/수능점수'], errors='coerce')

# 'view' 열의 값을 50으로 나눈 뒤 올림하여 1로 치환
filtered_df['가중치_조회수'] = np.ceil(filtered_df['view'] / 50)

# 'rounded_average'와 '3. 내신/수능점수' 열의 차이 계산
filtered_df['가중치_성적'] = filtered_df['3. 내신/수능점수'].apply(pd.to_numeric, errors='coerce')
filtered_df['가중치_성적'] = abs(filtered_df['가중치_성적'] - rounded_average)


# 1, 2, 3 열 값을 더하여 새로운 열 생성
filtered_df['합계'] = filtered_df['가중치_중요단어카운팅수'] + filtered_df['가중치_조회수'] 
#+ filtered_df['가중치_성적']

# 중요단어 카운팅 수와 no 열을 기준으로 중복 제거
filtered_df = filtered_df.sort_values(by=['합계'], ascending=False)
filtered_df = filtered_df.drop_duplicates(subset=['합계', 'no'], keep='first')



print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
print(f"[{user_input}에 게시글들에 대한 통합 결과입니다]")
print(f"{user_input}의 합격 수기를 작성한 이들의 내신 평균 점수:"+str(rounded_average))
print(f"{user_input}의 합격 수기를 작성한 이들의 고등학교 유형 분류 결과: ")
for school_type in school_types:
    count = type_counts.get(school_type, 0)
    percentage = (count / total_count) * 100
    print(f'{school_type}: {count}개 ({percentage:.2f}%)')
keywords()

print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
print("")
for i, row in enumerate(filtered_df.head(3).iterrows(), start=1):
    print(f"[{user_input}에 대한 {i}번째 게시글입니다.]")
    print(f"1. 글 제목: <{row[1]['title']}>")
    print(f"2. 작성 날짜: {row[1]['date']}")
    print(f"3. 출신 고교: {row[1]['2. 출신고교 종류']}")
    print(f"4. 내신 점수: {row[1]['3. 내신/수능점수']}")
    print(f"5. 입시 결과: \n{row[1]['new_col']}",end="")
    print(f"6. 스펙 핵심: {row[1]['스펙_핵심_추출']}")
    print(f"6. 스펙 요약: {row[1]['4. 요약']}")
    print(f"7. 후배들에게 전하고 싶은 말: {row[1]['5. 요약']}")
    print("")
    print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
    print("")


<>:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?


원하시는 대학명과 학과를 입력해주세요 ex)광운/정보융합 : 광운/정보융합
태깅된 행을 찾았습니다. 해당하는 열의 개수: 8



C:\Users\dlxod\AppData\Local\Temp\ipykernel_19324\980398444.py:89: DeprecationWarning: use options instead of chrome_options
  wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)
C:\Users\dlxod\AppData\Local\Programs\Python\Python310\lib\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')
C:\Users\dlxod\AppData\Local\Temp\ipykernel_19324\980398444.py:134: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if sentence is not '':


ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
[광운/정보융합에 게시글들에 대한 통합 결과입니다]
광운/정보융합의 합격 수기를 작성한 이들의 내신 평균 점수:3.33
광운/정보융합의 합격 수기를 작성한 이들의 고등학교 유형 분류 결과: 
일반고: 7개 (87.50%)
자사고: 0개 (0.00%)
특성화고: 0개 (0.00%)
과학고: 0개 (0.00%)
외고: 0개 (0.00%)
광운/정보융합 사람들의 스펙 핵심 키워드 : 
['기부', '대회', '파이썬', '코딩', '동아리', '학습', '컴퓨터', '봉사', '전공', '관련']
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

[광운/정보융합에 대한 1번째 게시글입니다.]
1. 글 제목: <광운대 정보융합 최초합 기록>
2. 작성 날짜: 2022.11.11.
3. 출신 고교: 일반고
4. 내신 점수: nan
5. 입시 결과: 
광운대/정보융합학부/소프트웨어우수전형
6. 스펙 핵심: []
6. 스펙 요약:   많아서 다 적지는 못하겠지만.. 123학년 생기부 대부분이 컴퓨터관련 활동으로 채워져있음 프로그래밍 프로젝트 10개 이상  
7. 후배들에게 전하고 싶은 말: 면접 얘기를 조금 하면 소프트웨어우수자전형이 활동 많이 물어볼줄 알았는데 면접때 활동 하나도 안물어봄. 시간이 되면 면접 내용 자세하게 글 적어둘게요 

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

[광운/정보융합에 대한 2번째 게시글입니다.]
1. 글 제목: <일반고 3.1 4합 수시 최종 기록 (최대한 자세하게 적었어요!)>
2. 작성 날짜: 2022.07.07.
3. 출신 고교: 일반고
4. 내신 점수: nan
5. 입시 결과: 
동국대학교 고른기회통합농어촌전형 AI 융합학부 - 1차 불합
서울과학기술대학교 첨단인재전형 인공지능

In [ ]:
# '중요단어 카운팅 수'기반 [결과 출력 창]
import pandas as pd

# 중요단어 카운팅 수와 no 열을 기준으로 중복 제거
filtered_df = filtered_df.sort_values(by=['중요단어 카운팅 수', 'no'], ascending=False)
filtered_df = filtered_df.drop_duplicates(subset=['중요단어 카운팅 수', 'no'], keep='first')

print(f"[{user_input}에 게시글들에 대한 통합 결과입니다]")
print(f"{user_input}의 합격 수기를 작성한 이들의 내신 평균 점수:"+rounded_average)
print(f"{user_input}의 합격 수기를 작성한 이들의 고등학교 유형 분류 결과: ")
for school_type in school_types:
    count = type_counts.get(school_type, 0)
    percentage = (count / total_count) * 100
    print(f'{school_type}: {count}개 ({percentage:.2f}%)')
keywords()

print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
print("")
for i, row in enumerate(filtered_df.head(3).iterrows(), start=1):
    print(f"[{user_input}에 대한 {i}번째 게시글입니다.]")
    print(f"1. 글 제목: <{row[1]['title']}>")
    print(f"2. 작성 날짜: {row[1]['date']}")
    print(f"3. 출신 고교: {row[1]['2. 출신고교 종류']}")
    print(f"4. 내신 점수: {row[1]['3. 내신/수능점수']}")
    print(f"5. 입시 결과: \n{row[1]['new_col']}",end="")
    print(f"6. 스펙 핵심: {row[1]['스펙_핵심_추출']}")
    print(f"6. 스펙 요약: {row[1]['4. 요약']}")
    print(f"7. 후배들에게 전하고 싶은 말: {row[1]['5. 요약']}")
    print("")
    print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
    print("")


★광운/정보융합 사람들의 스펙 핵심 키워드 : 
['기부', '대회', '파이썬', '코딩', '동아리', '학습', '컴퓨터', '봉사', '전공', '관련']
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

[광운/정보융합에 대한 1번째 게시글입니다.]
1. 글 제목: <일반고 3.1 4합 수시 최종 기록 (최대한 자세하게 적었어요!)>
2. 작성 날짜: .07.07.
3. 출신 고교: 일반고
4. 내신 점수: nan
5. 입시 결과: 
동국대학교 고른기회통합농어촌전형 AI 융합학부 - 1차 불합
서울과학기술대학교 첨단인재전형 인공지능응용학과 - 1차 불합
국민대학교 학생부종합농어촌전형 소프트웨어학부 - 예비 5번 2차 추합
세종대학교 학생부종합고른기회 인공지능학과 - 최초합
광운대학교 학생부종합농어촌전형 정보융합학부 - 최초합
가천대학교 학생부종합농어촌전형 인공지능전공 - 최초합
6. 스펙 핵심: ['생기부 18장']
6. 스펙 요약: nan
7. 후배들에게 전하고 싶은 말: 세특은 이런 식으로 진행하고 창체는 제가 프로젝트를 진행해서 결과물이 나올 수 있도록 노력한 것 같아요 입시 끝나고 생각해보면 교과 선생님들께 뜬금없이 가서 보고서 제출하고 생기부 관련해서 부탁도 많이 드리고...

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

[광운/정보융합에 대한 2번째 게시글입니다.]
1. 글 제목: <일반고 4.5 광운대 정보융합학부 최초합>
2. 작성 날짜: .11.12.
3. 출신 고교: 일반고
4. 내신 점수: 4.5
5. 입시 결과: 
광운대학교 / 정보융합학부 / 소프트웨어우수인재전형
6. 스펙 핵심: ['생기부 18장', '봉사 시간']
6. 스펙 요약: nan
7. 후배들에게 전하고 싶은 말: nan

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [ ]:
import pandas as pd
import re
import numpy as np

# 숫자 값 추출 함수 정의
def extract_number(value):
    try:
        number = re.findall(r'[+-]?\d+(?:\.\d+)?', str(value))
        if len(number) > 0:
            return float(number[0])
        else:
            return np.nan
    except ValueError:
        return np.nan

# 각 셀에 있는 숫자 값 추출하여 리스트에 저장
numbers = []
for value in filtered_df['3. 내신/수능점수']:
    number = extract_number(value)
    if not np.isnan(number):
        numbers.append(number)

# 평균 계산
average = np.mean(numbers)

# 평균 값을 소수점 둘째 자리에서 반올림하여 출력
# rounded_average 변수에 검색한 학교/학과 사람들의 평균 내신 점수 넣음.
rounded_average = round(average, 2)
print("평균:", rounded_average)


평균: 3.33


In [ ]:
import pandas as pd

# filtered_df 데이터프레임에서 학교 유형 분류 및 카운트
school_types = ['일반고', '자사고', '특성화고', '과학고', '외고']
total_count = len(filtered_df)
type_counts = filtered_df['2. 출신고교 종류'].value_counts()

# '외고'와 '외국어고'를 하나로 합치기
type_counts['외고'] = type_counts.get('외고', 0) + type_counts.get('외국어고', 0)
if '외국어고' in type_counts:
    del type_counts['외국어고']

# 분류 결과 출력
print('학교 유형 분류 결과:')
for school_type in school_types:
    count = type_counts.get(school_type, 0)
    percentage = (count / total_count) * 100
    print(f'{school_type}: {count}개 ({percentage:.2f}%)')


학교 유형 분류 결과:
일반고: 7개 (87.50%)
자사고: 0개 (0.00%)
특성화고: 0개 (0.00%)
과학고: 0개 (0.00%)
외고: 0개 (0.00%)
